<a href="https://colab.research.google.com/github/srlucasromulo/tp2-bioinspired/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install numpy
!pip -q install pandas
from typing import List
import numpy as np
import pandas as pd
import copy

#Load weighted directed graph

In [232]:
file_edges = pd.read_csv('https://raw.githubusercontent.com/srlucasromulo/tp2-bioinspired/main/edges', sep=';')
file_vertex = pd.read_csv('https://raw.githubusercontent.com/srlucasromulo/tp2-bioinspired/main/vertex', sep=';')

vertex = list(file_vertex['name'])  # list of vertex
edges = {v:{} for v in vertex}      # list of edges and its weights (distances)

# reads the edges from file
for edge in file_edges.iloc:
    vertex0 = edge['vertex0']
    vertex1 = edge['vertex1']
    distance = edge['distance']
    edges[vertex0][vertex1] = distance

#Set parameters

In [233]:
# the distance of a path
def objective_function(path):
    return sum([edges[path[i]][path[i + 1]] for i in range(len(path) - 1)])

n = 5               # population size
n_gen = 100         # n generations
stop_criteria = 10  # stop after n_gen w/o improvement

alpha = 8
beta = 5
ro = 0.5
Q = 5000

starvation_death = 50

objective_path = ['CTAN', 'CDB', 'CSA', 'CDB', 'CTAN']

#Ant and Colony classes

##Ant

In [234]:
def select_next_vertex(current, target, possibilities):
    if target in possibilities:
        return target

    def random_choice(possibilities):
        count = np.array([possibilities[key] for key in possibilities.keys()])
        count = 1 / count
        return np.random.choice(list(possibilities.keys()), p=count/sum(count))

    def pheromone_choice(possibilities):
        max_value = max([possibilities[key] for key in possibilities.keys()])
        vertex = []
        for key in possibilities.keys():
            if possibilities[key] == max_value:
                vertex.append(key)
        return np.random.choice(vertex)
        
    choice_method = \
        np.random.choice([pheromone_choice, random_choice], p=[alpha/(alpha+beta), beta/(alpha+beta)])
    return choice_method(possibilities)


class Ant:

    def __init__(self, vertex):
        self.fitness = np.inf
        self.vertex = vertex
        self.path = [vertex]
        self.starved = False

    def __str__(self):
        return f'{self.fitness}, {self.path}'

    def fitness_evaluate(self):
        self.fitness = objective_function(self.path)

    def deposit_pheromone(self, pheromone, factor):
        for i in range(len(self.path) - 1):
            pheromone[self.path[i]][self.path[i + 1]] += factor

    def generate_path(self, pheromone, alpha, beta):
        for target in objective_path[1:]:
            starvation = 0

            partial_path = [self.vertex]
            while self.vertex != target and not self.starved:
                possibilities = edges[self.vertex]

                next = select_next_vertex(self.vertex, target, possibilities)

                if next in partial_path:
                    index = partial_path.index(next)
                    partial_path = partial_path[:index]

                if next != target:
                    starvation = len(partial_path) + 1
                    if starvation == starvation_death:
                        self.starved = True
                
                self.vertex = next
                partial_path.append(next)

            if self.starved:
                break

            self.path += partial_path[1:]

##Colony

In [235]:
def initialize_pheromone():
    pheromone = {v:{} for v in vertex}
    for vertex0 in edges.keys():
        for vertex1 in edges[vertex0].keys():
            pheromone[vertex0][vertex1] = 10e-16
    return pheromone


def evaporate_all_edges_pheromone(pheromone):
    for vertex0 in edges.keys():
        for vertex1 in edges[vertex0].keys():
            pheromone[vertex0][vertex1] *= (1 - ro)


class Colony:

    def __init__(self):
        self.alpha = alpha
        self.beta = beta
        self.ants = []
        self.pheromone = initialize_pheromone()

    def __str__(self):
        string = ''
        for ant in self.ants:
            string += str(ant) + '\n'
        return string

    def initialize_ants(self):
        self.ants = [Ant(objective_path[0]) for _ in range(n)]

    def evaluate_ants(self):
        live_ants = [ant for ant in self.ants if not ant.starved]
        self.ants = live_ants
        for ant in self.ants:
            ant.fitness_evaluate()

    def generate_paths(self):
        for ant in self.ants:
            ant.generate_path(self.pheromone, self.alpha, self.beta)

    def update_pheromone(self):
        evaporate_all_edges_pheromone(self.pheromone)

        sorted(self.ants, key=lambda x: x.fitness)
        
        live_ants = len(self.ants)
        for i in range(live_ants):
            factor = (live_ants - i) * Q / self.ants[i].fitness
            self.ants[i].deposit_pheromone(self.pheromone, factor)

    def best_ant(self):
        if len(self.ants) == 0:
            return None

        index = np.argmin([ant.fitness for ant in self.ants])
        best = self.ants[index]
        return copy.deepcopy(best)

#ACO algorithm

In [236]:
def aco():
    colony = Colony()

    global_best = Ant(0)

    stop = 0
    for _ in range(n_gen):
        colony.initialize_ants()
        colony.generate_paths()
        colony.evaluate_ants()
        colony.update_pheromone()

        best = colony.best_ant()
        if best.fitness < global_best.fitness:
            global_best = copy.deepcopy(best)
            stop = 0

        if stop == stop_criteria:
            break
        else:
            stop += 1

    return global_best

#Exec script

In [238]:
data = []

global_best = Ant(0)

for alpha in [1, 2]:
    for beta in [1, 2]:
        for ro in [.3, .5, .7]:

            execs = []
            params_best = Ant(0)
            for c in range(5):
                print(alpha,beta,ro,c)

                best = aco()

                execs.append(best.fitness)

                if best.fitness < params_best.fitness:
                    params_best = copy.deepcopy(best)

                if best.fitness < global_best.fitness:
                    global_best = copy.deepcopy(best)
                
            new_entry = {
                'alpha': alpha,
                'beta': beta,
                'ro': ro,
                'fitness': params_best.fitness,
                'deviation': np.std(execs)
            }
            data.append(new_entry)

df = pd.DataFrame(data)
df.to_csv('execs')

with open('output', 'w') as file:
    file.write(str(global_best.fitness) + '\n')
    for vertex in global_best.path:
        file.write(vertex + '\n')

1 1 0.3 0
1 1 0.3 1
1 1 0.3 2
1 1 0.3 3
1 1 0.3 4
1 1 0.5 0
1 1 0.5 1
1 1 0.5 2
1 1 0.5 3
1 1 0.5 4
1 1 0.7 0
1 1 0.7 1
1 1 0.7 2
1 1 0.7 3
1 1 0.7 4
1 2 0.3 0
1 2 0.3 1
1 2 0.3 2
1 2 0.3 3
1 2 0.3 4
1 2 0.5 0
1 2 0.5 1
1 2 0.5 2
1 2 0.5 3
1 2 0.5 4
1 2 0.7 0
1 2 0.7 1
1 2 0.7 2
1 2 0.7 3
1 2 0.7 4
2 1 0.3 0
2 1 0.3 1
2 1 0.3 2
2 1 0.3 3
2 1 0.3 4
2 1 0.5 0
2 1 0.5 1
2 1 0.5 2
2 1 0.5 3
2 1 0.5 4
2 1 0.7 0
2 1 0.7 1
2 1 0.7 2
2 1 0.7 3
2 1 0.7 4
2 2 0.3 0
2 2 0.3 1
2 2 0.3 2
2 2 0.3 3
2 2 0.3 4
2 2 0.5 0
2 2 0.5 1
2 2 0.5 2
2 2 0.5 3
2 2 0.5 4
2 2 0.7 0
2 2 0.7 1
2 2 0.7 2
2 2 0.7 3
2 2 0.7 4
